In [9]:
#import library
import pandas as pd
import csv  #import or export csv, txt file
import math #use some function like ceil
import os   #access a directory, join path and file
import time

In [4]:
'''
python pandas dataframe dtypes property may not detect some datetime column because they use nonstandard format.
These column will be showed as object datatype, take use pd.to_datetime function to convert it, if it runs into error
it will be real string column
'''

def determine_Obj_Data_Type (df, file, column):
    try: 
        df[column] = df[column].apply(pd.to_datetime)  #try to convert to datetime
        return file, column, 'datetime', 'Null'
    except: 
        length = df[column].str.len().max()  #max lengh of the given column
        #if varchar length < 10 use it, or ceil it to next decimal (give 20% extra length)
        return file, column, 'varchar', length if length <= 10 else math.ceil(length*.12)*10  


In [5]:
'''
this function will pass in a flat file folder. current support file in csv and txt format (use comma, tab separator file)
loop thrugh the flat file, import flat file into a data frame
loop through give flat file's column to check data type
if start with int, it is int 
if start with float, it is float
if it is object, then will call determine_obj_data_type to make decision
'''

def getFlatFileMetaDate(source_folder, output_file_path):
    files = os.listdir(source_folder)
    textfiles = [file for file in files if file.endswith('.csv') or file.endswith('.txt')] # filter csv and txt file
    with open (output_file_path, 'w', newline='') as outfile:
        writer = csv.writer(outfile)
        writer.writerow(('file', 'column', 'data type', 'length')) #write header
        for file in textfiles:
            #if csv use comma, txt use \t as delimiter
            df_file = pd.read_csv(os.path.join(source_folder, file), delimiter = None if file.endswith('csv') else '\t') 
            df = df_file.dtypes.to_frame().reset_index()  #no column header but in column, data type format 
            df.columns = ['column', 'data_type'] #rename column header
            for index, row in df.iterrows():
                #print(row['column'], row['data_type'])
                if str(row['data_type']).startswith('int'):  #handle int, like int32, int64
                    writer.writerow(( file, row['column'], 'Int', 'Null'))
                elif str(row['data_type']).startswith('float'):
                    writer.writerow(( file, row['column'],'Float', 'Null'))
                elif str(row['data_type']) ==('object'):
                    writer.writerow ( determine_Obj_Data_Type(df_file, file, row['column'] ))


In [10]:
start_time = time.time()
root = r'C:\temp\testdata'
result = r'C:\Users\Wenlei\Documents\meta_data_extraction\final_meta_data.csv'
getFlatFileMetaDate (root, result)
print("---Process used %s seconds ---" % (time.time() - start_time))

---Process used 4.778866291046143 seconds ---
